In [1]:
import sys
print(sys.version)
import seaborn as sns
import statistics as st
from statsmodels.graphics.gofplots import qqplot
import matplotlib.pyplot as plt
import math as m
from scipy.stats import shapiro
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd
import operator
from fitter import Fitter, get_common_distributions, get_distributions
colors = ['thistle','lightblue','lightgreen','yellow','pink']

3.8.8 (default, Apr 13 2021, 12:59:45) 
[Clang 10.0.0 ]


In [2]:
# ways to create a metric between those values
# - calculate how many instances are above the established 
# - based on the overall length 


# based on the cluster analysis
# take the overall noise and put in the distribution between 1 and 10 based on its mean value and variance;
# for noise the complement for one to 10;
# for length, smaller lengths corresponds to at least majority of the the clusters -> compared that to 1 t0 10;
# for coverage similar effect, scale to 1 to 10, how many of those alignments are more than the identiy they claimed;

# for the tuples
# similar concepts based considering the amount of tuples and max cliques;
# increase its values based on the same behaviour in smaller cliques, but in a smaller proportion;

# double penalizing the instances that fall below 50% alpha safe

In [3]:
def readInputs(filefolder):
    
    data_cc = pd.read_csv(filefolder+'cc.csv',usecols=['90', '70', '40', '30', '20'],sep=';',decimal=",")
    data_cov = pd.read_csv(filefolder+'coverage.csv',usecols=['90', '70', '40', '30', '20'],sep=';',decimal=",")
    data_lc = pd.read_csv(filefolder+'lc.csv',usecols=['90', '70', '40', '30', '20'],sep=';')
    data_nc = pd.read_csv(filefolder+'nc.csv',usecols=['90', '70', '40', '30', '20'],sep=';')
    data_length = pd.read_csv(filefolder+'length.csv',usecols=['90', '70', '40', '30', '20'],sep=';')
    data_noise = pd.read_csv(filefolder+'noise.csv',usecols=['90', '70', '40', '30', '20'],sep=';')
    data_tuple = pd.read_csv(filefolder+'noise.csv',usecols=['90', '70', '40', '30', '20'],sep=';')
    data_max = pd.read_csv(filefolder+'noise.csv',usecols=['90', '70', '40', '30', '20'],sep=';')
    
    #data_numberOfTuples
    #data_numberOfMaxCliques
    
    data ={
        'cc': data_cc,
        'cov': data_cov,
        'lc': data_lc,
        'nc': data_nc,
        'length': data_length,
        'noise': data_noise,
        'tuples': data_tuple,
        'maxcliques': data_max,
        'overall': [0,0,0,0,0]
        
    }
    
    return data

In [4]:
def lengthMetric(data):
    
    dataLength = data['length']
    score = data['overall']
    
    len_90 = dataLength['90']
    len_70 = dataLength['70']
    len_40 = dataLength['40']
    len_30 = dataLength['30']
    len_20 = dataLength['20']
    
    for i in range(0,len(len_90)):
        score[0] += (len_90[i]/max(len_90))/10
        score[1] += (len_70[i]/max(len_70))/10
        score[2] += (len_40[i]/max(len_40))/10
        score[3] += (len_30[i]/max(len_30))/10
        score[4] += (len_20[i]/max(len_20))/10
    
    data['overall'] = score
    
    return data;

In [5]:
def coverageMetric(data):
    
    dataCoverage = data['cov']
    score = data['overall']
    
    cov_90 = dataCoverage['90']
    cov_70 = dataCoverage['70']
    cov_40 = dataCoverage['40']
    cov_30 = dataCoverage['30']
    cov_20 = dataCoverage['20']

    
    for i in range(0,len(cov_90)):
        if float(cov_90[i]) >= 0.9:
            score[0] += 1/10
        if float(cov_70[i]) >= 0.7:
            score[1] += 1/10
        if float(cov_40[i]) >= 0.4:
            score[2] += 1/10
        if float(cov_30[i]) >= 0.3:
            score[3] += 1/10
        if float(cov_20[i]) >= 0.2:
            score[4] += 1/10
            
    data['overall'] = score
    
    return data


In [6]:
def noiseMetric(data):
    
    dataNoise = data['noise']
    
    noise_90 = dataNoise['90']
    noise_70 = dataNoise['70']
    noise_40 = dataNoise['40']
    noise_30 = dataNoise['30']
    noise_20 = dataNoise['20']
    score = data['overall']
    
    for i in range(0,len(noise_90)):
        score[0] += (1 - noise_90[i])/10
        score[1] += (1 - noise_70[i])/10
        score[2] += (1 - noise_40[i])/10
        score[3] += (1 - noise_30[i])/10
        score[4] += (1 - noise_20[i])/10
    
    data['overall'] = score
    
    return data

In [7]:
def noiseTupleMetric(data):
    
    dataNoise = data['nc']
    dataTuples = data['tuples']
    dataMax = data['maxcliques']
    score = data['overall']
    
    noise_90 = dataNoise['90']
    tuple_90 = dataTuples['90']
    max_90 = dataMax['90']
    noise_70 = dataNoise['70']
    tuple_70 = dataTuples['70']
    max_70 = dataMax['70']
    noise_40 = dataNoise['40']
    tuple_40 = dataTuples['40']
    max_40 = dataMax['40']
    noise_30 = dataNoise['30']
    tuple_30 = dataTuples['30']
    max_30 = dataMax['30']
    noise_20 = dataNoise['20']
    tuple_20 = dataTuples['20']
    max_20 = dataMax['20']
    
    for i in range(0,len(noise_90)):
        noise_90[i] = float(noise_90[i])*float(tuple_90[i]) + (float(noise_90[i]))*float(max_90[i])
        noise_70[i] = float(noise_70[i])*float(tuple_70[i]) + (float(noise_70[i]))*float(max_70[i])
        noise_40[i] = float(noise_40[i])*float(tuple_40[i]) + (float(noise_40[i]))*float(max_40[i])
        noise_30[i] = float(noise_30[i])*float(tuple_30[i]) + (float(noise_30[i]))*float(max_30[i])
        noise_20[i] = float(noise_20[i])*float(tuple_20[i]) + (float(noise_20[i]))*float(max_20[i])
    
    for i in range(0,len(noise_90)):
        score[0] += (1-noise_90[i])/10
        score[1] += (1-noise_70[i])/10
        score[2] += (1-noise_40[i])/10
        score[3] += (1-noise_30[i])/10
        score[4] += (1-noise_20[i])/10
    
    data['overall'] = score
    
    return data

In [8]:
def lengthTupleMetric(data):
    
    dataLength = data['lc']
    dataTuples = data['tuples']
    dataMax = data['maxcliques']
    score = data['overall']
    
    len_90 = dataLength['90']
    tuple_90 = dataTuples['90']
    max_90 = dataMax['90']
    len_70 = dataLength['70']
    tuple_70 = dataTuples['70']
    max_70 = dataMax['70']
    len_40 = dataLength['40']
    tuple_40 = dataTuples['40']
    max_40 = dataMax['40']
    len_30 = dataLength['30']
    tuple_30 = dataTuples['30']
    max_30 = dataMax['30']
    len_20 = dataLength['20']
    tuple_20 = dataTuples['20']
    max_20 = dataMax['20']
    
    for i in range(0,len(len_90)):
        len_90[i] = float(len_90[i])*float(tuple_90[i]) + (float(len_90[i]))*float(max_90[i])
        len_70[i] = float(len_70[i])*float(tuple_70[i]) + (float(len_70[i]))*float(max_70[i])
        len_40[i] = float(len_40[i])*float(tuple_40[i]) + (float(len_40[i]))*float(max_40[i])
        len_30[i] = float(len_30[i])*float(tuple_30[i]) + (float(len_30[i]))*float(max_30[i])
        len_20[i] = float(len_20[i])*float(tuple_20[i]) + (float(len_20[i]))*float(max_20[i])

    
    for i in range(0,len(len_90)):
        score[0] += (len_90[i]/max(len_90))/10
        score[1] += (len_70[i]/max(len_70))/10
        score[2] += (len_40[i]/max(len_40))/10
        score[3] += (len_30[i]/max(len_30))/10
        score[4] += (len_20[i]/max(len_20))/10
            
    data['overall'] = score
    
    return data

In [9]:
def coverageTupleMetric(data):
    
    dataCoverage = data['cc']
    dataTuples = data['tuples']
    dataMax = data['maxcliques']
    score = data['overall']
    
    cov_90 = dataCoverage['90']
    tuple_90 = dataTuples['90']
    max_90 = dataMax['90']
    cov_70 = dataCoverage['70']
    tuple_70 = dataTuples['70']
    max_70 = dataMax['70']
    cov_40 = dataCoverage['40']
    tuple_40 = dataTuples['40']
    max_40 = dataMax['40']
    cov_30 = dataCoverage['30']
    tuple_30 = dataTuples['30']
    max_30 = dataMax['30']
    cov_20 = dataCoverage['20']
    tuple_20 = dataTuples['20']
    max_20 = dataMax['20']
    
    for i in range(0,len(cov_90)):
        cov_90[i] = float(cov_90[i])*float(tuple_90[i]) + (1 - float(cov_90[i]))*float(max_90[i])
        cov_70[i] = float(cov_70[i])*float(tuple_70[i]) + (1 - float(cov_70[i]))*float(max_70[i])
        cov_40[i] = float(cov_40[i])*float(tuple_40[i]) + (1 - float(cov_40[i]))*float(max_40[i])
        cov_30[i] = float(cov_30[i])*float(tuple_30[i]) + (1 - float(cov_30[i]))*float(max_30[i])
        cov_20[i] = float(cov_20[i])*float(tuple_20[i]) + (1 - float(cov_20[i]))*float(max_20[i])
    
    for i in range(0,len(cov_90)):
        cov_90[i] = cov_90[i]/max(cov_90)
        cov_70[i] = cov_70[i]/max(cov_70)
        cov_40[i] = cov_40[i]/max(cov_40)
        cov_30[i] = cov_30[i]/max(cov_30)
        cov_20[i] = cov_20[i]/max(cov_20)

    
    for i in range(0,len(cov_90)):
        if float(cov_90[i]) >= 0.9:
            score[0] += 1/10
        if float(cov_70[i]) >= 0.7:
            score[1] += 1/10
        if float(cov_40[i]) >= 0.4:
            score[2] += 1/10
        if float(cov_30[i]) >= 0.3:
            score[3] += 1/10
        if float(cov_20[i]) >= 0.2:
            score[4] += 1/10
            
    data['overall'] = score
    
    return data

In [10]:
filefolder = ['DataBackup/Backup-TX/','DataBackup/Backup-HPI/','DataBackup/Backup-SIM/','DataBackup/Backup-HL/']

for i in range(0,len(filefolder)):
    data = readInputs(filefolder[i])
    data = lengthMetric(data)
    data = coverageMetric(data)
    data = noiseMetric(data)
    data = coverageTupleMetric(data)
    data = lengthTupleMetric(data)
    data = noiseTupleMetric(data)

    print(data['overall'])

# metrics for length



[32.69582354289866, 29.45974033306384, 30.833733354320742, 27.82334001618863, 27.548689032160066]
[33.65879720753293, 32.09245538649122, 34.3547902208835, 33.71641803467384, 34.076869504356026]
[33.65879720753293, 32.09245538649122, 34.3547902208835, 33.71641803467384, 34.076869504356026]
[32.59113597994025, 30.44873162734793, 32.16839636537102, 32.12721453392271, 30.54275770532225]
